In [1]:
import pandas as pd
docs = pd.read_json('docs_with_embeddings_non_outliers.json', lines=True)
reviews  =pd.read_json('filtered_dataset/selected_users.json', lines=True)

# Contruindo data frame com usuario | produto consumido | nota
consumes = reviews.join(docs.set_index("asin"), on="asin", how="inner")
consumes = consumes[['reviewerID', 'asin', 'vec', 'topic_probs_vec', 'overall', 'topic']]
consumes

,reviewerID,asin,vec,topic_probs_vec,overall,topic
0,AZB4CQ9JZSUQB,0005019281,"[-0.010102539300000001, 0.0370328091, 0.047199...","[0.007562089100000001, 0.0096547827, 0.0062892...",5,15
1,A101IGU6UDKW3X,0005019281,"[-0.010102539300000001, 0.0370328091, 0.047199...","[0.007562089100000001, 0.0096547827, 0.0062892...",3,15
2,A1L2DLWYRNHKDR,0005019281,"[-0.010102539300000001, 0.0370328091, 0.047199...","[0.007562089100000001, 0.0096547827, 0.0062892...",2,15
3,A3I4C4LS3ID7Y4,0005019281,"[-0.010102539300000001, 0.0370328091, 0.047199...","[0.007562089100000001, 0.0096547827, 0.0062892...",4,15
4,A3ISFNQ9FJSPFP,0005019281,"[-0.010102539300000001, 0.0370328091, 0.047199...","[0.007562089100000001, 0.0096547827, 0.0062892...",4,15
...,...,...,...,...,...,...
224759,A34PUCLNFN62D3,B005LAIHZE,"[-0.0778384507, 0.0128778582, -0.0573381819, -...","[0.38536024010000003, 0.018467520600000002, 0....",1,0
224760,A3HL4CWBBPL1WW,B005LAII1C,"[-0.0867156088, -0.0997771174, -0.0006877516, ...","[0.0068604903, 0.0075558216, 0.005630908400000...",4,8
224761,A22R1X2P28BFCV,B005LAIHY0,"[-0.0056930352, -0.0479681119, -0.0175523479, ...","[1.2009330929999999e-307, 1.0, 1.1486361359999...",4,1
224762,A20ZCHB9EV90D4,B005LAII1C,"[-0.0867156088, -0.0997771174, -0.0006877516, ...","[0.0068604903, 0.0075558216, 0.005630908400000...",5,8


In [2]:
import numpy as np

train = consumes

def mark_train_test(group, frac=0.7):
    n = len(group)
    n_train = int(np.floor(frac * n))
    idx_train = group.sample(n=n_train, random_state=42).index
    group['set'] = np.where(group.index.isin(idx_train), 'train', 'test')
    return group

def mean_vec(series):
    arrs = [np.asarray(v) for v in series]
    return np.mean(np.stack(arrs, axis=0), axis=0)

train = train.groupby('reviewerID', group_keys=False).apply(mark_train_test)

consumes = train

test = train[train['set'] == 'test']
train = train[train['set'] == 'train']

train

train_users = train[['reviewerID', 'topic_probs_vec']].groupby("reviewerID", as_index=False).agg(
        topic_probs_vec=('topic_probs_vec', mean_vec)
    )

train_users

/tmp/ipykernel_41892/3132156902.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = train.groupby('reviewerID', group_keys=False).apply(mark_train_test)


,reviewerID,topic_probs_vec
0,A10175AMUHOQC4,"[0.029975303245238095, 0.023132877104761904, 0..."
1,A101IGU6UDKW3X,"[0.012343711674999999, 0.024682151139285714, 0..."
2,A103KNDW8GN92L,"[0.022816977961904763, 0.006555879095238095, 0..."
3,A106016KSI0YQ,"[0.05087469872931033, 0.023827831260344823, 0...."
4,A106YXO3EHVD3J,"[0.06177859683137255, 0.022587816470588242, 0...."
...,...,...
2273,AZC562U18BK2S,"[0.015800347795652175, 0.07973784546956521, 0...."
2274,AZDH08P9ZMWKJ,"[0.009129016638888888, 0.007020222294444444, 0..."
2275,AZDVOFC2MTIM5,"[0.010495319014285713, 0.00810307038214286, 0...."
2276,AZQ8EITRKV9GS,"[0.10122658025714286, 0.013437444516666666, 0...."


In [3]:
prods = docs[['asin', 'topic_probs_vec']]

In [4]:
from sklearn.neighbors import NearestNeighbors

user_matrix = np.vstack(train_users['topic_probs_vec'].to_numpy())
prods_matrix = np.vstack(prods['topic_probs_vec'].to_numpy())

prods_ids = prods['asin'].values

K = 100

nn = NearestNeighbors(
    n_neighbors=K,
    metric='cosine',
    n_jobs=-1
)

nn.fit(prods_matrix)

dist, ids = nn.kneighbors(user_matrix, return_distance=True)
sim = 1.0 - dist

rows = []
i = 0
for u in train_users['reviewerID'].values:

    j = 0
    for prod_id in ids[i]:
        asin = prods_ids[prod_id]
        score = sim[i][j]
        rows.append([u, asin, score])
        j +=1
    
    i += 1

scores_df = pd.DataFrame(rows, columns=['reviewerID', 'asin', 'score'])
scores_df = scores_df.merge(consumes[['reviewerID', 'asin', 'set']], on=['reviewerID', 'asin'], how="left")
scores_df

,reviewerID,asin,score,set
0,A10175AMUHOQC4,B0000U0X1Q,0.802153,NaN
1,A10175AMUHOQC4,6304558422,0.800647,NaN
2,A10175AMUHOQC4,B000JJ27BC,0.798811,NaN
3,A10175AMUHOQC4,B000FEILBG,0.795301,NaN
4,A10175AMUHOQC4,B001TKNWVO,0.795192,NaN
...,...,...,...,...
228470,AZSN1TO0JI87B,B00005JLHF,0.878867,NaN
228471,AZSN1TO0JI87B,B0001XOLII,0.878685,NaN
228472,AZSN1TO0JI87B,B003HFUVQI,0.878544,test
228473,AZSN1TO0JI87B,0792146433,0.878531,NaN


In [5]:
scores_df[scores_df['set'] == 'test']

,reviewerID,asin,score,set
192,A101IGU6UDKW3X,B004A8ZWVU,0.620733,test
488,A106YXO3EHVD3J,B004ZBAB3Q,0.701730,test
498,A106YXO3EHVD3J,B00029LO0Y,0.699117,test
499,A106YXO3EHVD3J,B005ELMC1U,0.699054,test
503,A1074GBGMK6136,B001GCUO16,0.837282,test
...,...,...,...,...
228281,AZQ8EITRKV9GS,B0021L8UG2,0.849477,test
228343,AZQ8EITRKV9GS,B000VKN1XM,0.843620,test
228367,AZQ8EITRKV9GS,6305785252,0.841760,test
228392,AZSN1TO0JI87B,B0011G6FDI,0.883433,test


In [8]:
import numpy as np
import pandas as pd

def ndcg_at_k(df, k=10):
    """
    Calcula nDCG@k médio entre usuários, considerando apenas:
      - Itens do 'test' como relevantes (1)
      - Itens com NaN no 'set' como não relevantes (0)
      - Itens do 'train' são ignorados
    """
    users = df['reviewerID'].unique()
    ndcgs = []

    for uid in users:
        user_df = df[df['reviewerID'] == uid].copy()

        # ignora itens que estavam no treino
        user_df = user_df[user_df['set'] != 'train']

        if user_df.empty:
            continue

        # define relevância binária: 1 se 'test', 0 se NaN
        user_df['relevance'] = (user_df['set'] == 'test').astype(int)

        # ordena pelas pontuações previstas
        user_df = user_df.sort_values(by='score', ascending=False)

        # pega o top-k do ranking
        top_k = user_df.head(k)
        rels = top_k['relevance'].to_numpy()

        # calcula DCG@k
        dcg = np.sum((2**rels - 1) / np.log2(np.arange(2, len(rels) + 2)))

        # calcula IDCG@k (ranking ideal)
        ideal_rels = np.sort(user_df['relevance'].to_numpy())[::-1][:k]
        idcg = np.sum((2**ideal_rels - 1) / np.log2(np.arange(2, len(ideal_rels) + 2)))

        ndcg = dcg / idcg if idcg > 0 else 0
        ndcgs.append(ndcg)

    return np.mean(ndcgs) if ndcgs else 0

# Exemplo de uso:
mean_ndcg = ndcg_at_k(scores_df, k=5)
print(f"nDCG@100 médio: {mean_ndcg:.4f}")

nDCG@100 médio: 0.0159
